In [ ]:
from pathlib import Path
import sys

lib_path = Path('../../').resolve()
sys.path.append(str(lib_path))
print('lib_path path: ', lib_path)

In [ ]:
import yaml

import pandas as pd

from cropscheduler.crops import CropModels
from cropscheduler.utils.weather import get_weather_df

csv_fp =  Path('/Users/yoonsj/Downloads') / 'Untitled.csv'
yaml_fp = Path('..') / 'default_parameters.yaml'

print(csv_fp.resolve())

weather_df = pd.read_csv(csv_fp)
with open(yaml_fp, 'r', encoding='utf-8') as f_param:
    DEFAULT_PARAMETERS = yaml.load(f_param, Loader=yaml.FullLoader)
crop_models = {ModelClass.key: ModelClass for ModelClass in CropModels}


### 쌀
  - 안전출수기
    - 출수일 ~ 출수후 40일 동안 평균기온 21 ~ 24 일 것

In [ ]:
crop_model = crop_models.get('middleLateMatureRice')()
crop_model.set_parameters(DEFAULT_PARAMETERS)
crop_model.set_weather_data(weather_df)
crop_model.set_start_doy()

start_doy = crop_model.start_doy
end_doy = crop_model.end_doy


In [ ]:
df = crop_model.weather_df.copy()
ref = crop_model.calculate_first_priority_params()
heading_range = ref.get('heading_range')

window_size = 40

df['tavg_40days'] = 0.5 * (df['tmax'] + df['tmin'])
_df = df.groupby('year')\
    .rolling(window_size)\
    .agg({'tavg_40days': 'mean', 'doy': 'min'})\
    .shift(-window_size+1)

heading_df = _df.query((
    f'doy >= {heading_range[0]} &' 
    f'doy <= {heading_range[1]}'
))

safe_heading_df = heading_df.query((
    '21 <= tavg_40days & tavg_40days <= 24'
))

safe_heading_prob = len(safe_heading_df) / len(heading_df) * 100
safe_heading_prob = round(safe_heading_prob, 1)
safe_heading_prob

### 배추
  - 한계온도 & 노출일수
    - high_extrema_temperature = 30
    - high_extrema_exposure_days = 5
    - low_extrema_temperature = 5
    - low_extrema_exposure_days = 5

In [ ]:
crop_model = crop_models.get('springCabbage')()
crop_model.set_parameters(DEFAULT_PARAMETERS)
crop_model.set_weather_data(weather_df)
crop_model.set_start_doy()

start_doy = crop_model.start_doy
end_doy = crop_model.end_doy


In [ ]:
df = crop_model.weather_df.copy()

high_extrema_temperature = 25  # DEV
low_extrema_temperature = 5  # DEV
high_extrema_exposure_days = 5  # DEV
low_extrema_exposure_days = 5  # DEV


In [ ]:
n_years = n_years = df['year'].nunique()

tmax_df = df.copy()[['year', 'doy', 'tmax']].sort_values(['year', 'doy'])\
            .query((f'tmax > {high_extrema_temperature} & '
                    f'doy >= {start_doy} & doy <= {end_doy}'))
tmax_df['doy_group'] = (tmax_df['doy'].diff(1).fillna(1) != 1).cumsum()
tmax_df['too_much_exposure'] = False
for _, group in tmax_df.groupby('doy_group'):
    if len(group) >= high_extrema_exposure_days:
        tmax_df.loc[group.index, 'too_much_exposure'] = True

_df = tmax_df.pivot(index='doy', columns='year', values='too_much_exposure').fillna(False).sort_index()
tmax_exposure_df = _df.sum(axis=1) / n_years

tmax_exposure_df